### 预热-模型存储
```
虽然tf.train.Saver()可以存储graph和session，但是没法支持存储后直接run_as_server，可能因为接口上没支持input_fn的接口设计
```

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# 通过Flags管理全局变量
tf.app.flags.DEFINE_integer('training_iteration', 10000, 'number of training iterations.')
tf.app.flags.DEFINE_integer('model_version', 1, 'version number of the model.')
tf.app.flags.DEFINE_string('work_dir', '/data/mnist', 'data directory.')
FLAGS = tf.app.flags.FLAGS

#加载数据
data_sets = input_data.read_data_sets(FLAGS.work_dir, one_hot=True)

Extracting /data/mnist/train-images-idx3-ubyte.gz
Extracting /data/mnist/train-labels-idx1-ubyte.gz
Extracting /data/mnist/t10k-images-idx3-ubyte.gz
Extracting /data/mnist/t10k-labels-idx1-ubyte.gz


In [2]:
import sys,os

#构建模型
with tf.Session() as sess:
  # placeholder 模型的输入
  serialized_tf_example = tf.placeholder(tf.string, name='tf_example')

  # 根据输入，解析出x特征
  feature_configs = {'x': tf.FixedLenFeature(shape=[784], dtype=tf.float32),}
  tf_example = tf.parse_example(serialized_tf_example, feature_configs)

  x = tf.identity(tf_example['x'], name='x')  # use tf.identity() to assign name

  # y是标签
  y_ = tf.placeholder('float', shape=[None, 10])
    
  # 模型相关参数
  w = tf.Variable(tf.zeros([784, 10]))
  b = tf.Variable(tf.zeros([10]))
  y = tf.nn.softmax(tf.matmul(x, w) + b, name='y')
  cross_entropy = -tf.reduce_sum(y_ * tf.log(y))
  train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
  values, indices = tf.nn.top_k(y, 10)
  table = tf.contrib.lookup.index_to_string_table_from_tensor(
      tf.constant([str(i) for i in xrange(10)]))
  prediction_classes = table.lookup(tf.to_int64(indices))
  correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))

  sess.run(tf.global_variables_initializer())
  for _ in range(FLAGS.training_iteration):
    batch = data_sets.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch[0], y_: batch[1]})
  print 'training accuracy %g' % sess.run(
      accuracy, feed_dict={x: data_sets.test.images,
                           y_: data_sets.test.labels})
  print 'Done training!'

  export_path_base = "/tmp/mnist_model"
  export_path = os.path.join(
      tf.compat.as_bytes(export_path_base),
      tf.compat.as_bytes(str(FLAGS.model_version)))
  print 'Exporting trained model to', export_path
  builder = tf.saved_model.builder.SavedModelBuilder(export_path)

  # Build the signature_def_map.
  classification_inputs = tf.saved_model.utils.build_tensor_info(
      serialized_tf_example)
  classification_outputs_classes = tf.saved_model.utils.build_tensor_info(
      prediction_classes)
  classification_outputs_scores = tf.saved_model.utils.build_tensor_info(values)

  classification_signature = (
      tf.saved_model.signature_def_utils.build_signature_def(
          inputs={
              tf.saved_model.signature_constants.CLASSIFY_INPUTS:
                  classification_inputs
          },
          outputs={
              tf.saved_model.signature_constants.CLASSIFY_OUTPUT_CLASSES:
                  classification_outputs_classes,
              tf.saved_model.signature_constants.CLASSIFY_OUTPUT_SCORES:
                  classification_outputs_scores
          },
        method_name=tf.saved_model.signature_constants.CLASSIFY_METHOD_NAME))

  tensor_info_x = tf.saved_model.utils.build_tensor_info(x)
  tensor_info_y = tf.saved_model.utils.build_tensor_info(y)

  prediction_signature = (
      tf.saved_model.signature_def_utils.build_signature_def(
          inputs={'images': tensor_info_x},
          outputs={'scores': tensor_info_y},
          method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME))

  legacy_init_op = tf.group(tf.tables_initializer(), name='legacy_init_op')
  builder.add_meta_graph_and_variables(
      sess, [tf.saved_model.tag_constants.SERVING],
      signature_def_map={
          'predict_images':
              prediction_signature,
          tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:
              classification_signature,
      },
      legacy_init_op=legacy_init_op)

  builder.save()

training accuracy 0.9164
Done training!
Exporting trained model to /tmp/mnist_model/1
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /tmp/mnist_model/1/saved_model.pb


### 1. Estimator使用
1. 使用现有的模型训练
2. 保存模型
3. 运行模型 server方式

SyntaxError: invalid syntax (<ipython-input-3-6cf82d865f33>, line 1)

### 2. 自建Estimator